In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import os
import copy

In [25]:
def load_data(train_batch_size, test_batch_size):
    # Fetch training data: total 60000 samples
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train = True, download=True,
                       transform=transforms.Compose([
                           transforms.Resize((32, 32)),
                           transforms.ToTensor()
                       ])),
        batch_size = train_batch_size, shuffle=True)

    # Fetch test data: total 10000 samples
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train = False, transform=transforms.Compose([
            transforms.Resize((32, 32)),
            transforms.ToTensor()
        ])),
        batch_size = test_batch_size, shuffle=True)

    return (train_loader, test_loader)

In [26]:
Batch_List=[505,1000,2500,5000,7000]


In [27]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # flatten as one dimension
        x = x.view(x.size()[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        x = self.fc3(x)
        return x

In [28]:
def train(model,num_epochs,train_load):
    model.train()
    train_load=train_load

    n_total_steps = len(train_load)
    train_losses = []
    train_epoch = []
    train_acc = []
    not_converged =True
    epoch = 0
    sensitivity=[]
    while not_converged:
        epoch += 1
        n_correct = 0
        n_samples = 0
        lossSum =0
        for i, (imgs, targets) in enumerate(train_load):

            imgs, targets = Variable(imgs),Variable(targets)

            optimizer.zero_grad()


            prediction = model(imgs)

            imgs.requires_grad = True

            loss = loss_function(prediction, targets)
            lossSum += loss

            # Backward and optimize
            loss.backward()
            optimizer.step()

            # sensitivity
            froGrad=0
            count =0


            for p in model.parameters():
                grad = 0.0
                if p.grad is not None:
                    grad = p.grad
                    froGrad_norm = torch.linalg.norm(grad).numpy()
                    froGrad += froGrad_norm
                    count += 1

            sensitivity.append(froGrad/count)




            _, predicted = torch.max(prediction.data, 1)
            n_samples += targets.size(0)
            n_correct += (predicted == targets).sum().item()
            acc = 100.0 * n_correct / n_samples

            train_losses.append(loss.item())
            train_acc.append(acc)
            train_epoch.append(epoch)


            if (i+1) % 20 == 0:
                print (f'Train O/P: Epoch [{epoch}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}',end= '\r',flush = True)
                if epoch == num_epochs:
                        #print("Max Epoch Reached")
                        not_converged = False
                elif (epoch > 5) and  (train_losses[-1] < 0.001):
                    if abs(train_losses[-3] - train_losses[-2]) < 1.0e-05 and abs(train_losses[-2] - train_losses[-1]) < 1.0e-05:
                        print("Convergeance reached for loss:",train_losses[-1])
                        not_converged = False

        trainAvgLoss = lossSum/i+1
        print("Avg Training loss:",trainAvgLoss)

    return train_epoch,train_losses,train_acc,sensitivity,trainAvgLoss

In [29]:

loss_function=nn.CrossEntropyLoss()

In [30]:
def test(model,test_load):
    test_load = test_load
    model.eval()
    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        testLoss = 0
        count = 0
        for imgs,targets in test_load:
            imgs, targets = Variable(imgs),Variable(targets)

            prediction = model(imgs)
            testLoss += loss_function(prediction,targets).item()
            # max returns (value ,index)
            _, predicted = torch.max(prediction.data, 1)
            n_samples += targets.size(0)
            n_correct += (predicted == targets).sum().item()
            count += 1
    netTest_loss = testLoss/count
    netTest_acc1 = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the test images: {netTest_acc1}% & Test Loss: {netTest_loss} ',end= '\r',flush = True)
    return netTest_acc1, netTest_loss

In [31]:
modelsTrainEpoch1 = []
modelsTrainLoss1 = []
modelsTrainAcc1 = []
modelsTestLoss1 = []
modelsTestAcc1 = []
modelsSensitivity1 = []

In [32]:
tempModel = Model ()

a=[]
for k in tempModel.parameters():
    a.append(torch.numel(k))

print(f'Total no of parameters in Model :{np.sum(a)}')

Total no of parameters in Model :61706


In [33]:
for i in range (len(Batch_List)):
    torch.manual_seed(1)

    j=copy.deepcopy(i)
    j = Model()
    #loss_function = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(j.parameters(), lr=1e-3, weight_decay= 1e-4)
    train_batch_size = int(Batch_List[i])
    test_batch_size = int(Batch_List[i])
    train_load,test_load= load_data(train_batch_size,test_batch_size)
    max_epochs = 6
    
    
    train_epoch,train_losses,train_acc,sensitivity,tvgLoss = train(j,max_epochs,train_load)
    testAcc, testLoss = test(j,test_load)
    
    modelsTrainEpoch1.append(train_epoch)
    modelsTrainLoss1.append(train_losses)
    modelsTrainAcc1.append(train_acc)
    modelsTestAcc1.append(testAcc)
    modelsTestLoss1.append(testLoss)
    modelsSensitivity1.append(sensitivity)

Avg Training loss: tensor(1.7810, grad_fn=<AddBackward0>)
Avg Training loss: tensor(1.1995, grad_fn=<AddBackward0>)
Avg Training loss: tensor(1.1260, grad_fn=<AddBackward0>)
Avg Training loss: tensor(1.0953, grad_fn=<AddBackward0>)
Avg Training loss: tensor(1.0801, grad_fn=<AddBackward0>)
Avg Training loss: tensor(1.0666, grad_fn=<AddBackward0>)
Avg Training loss: tensor(2.1748, grad_fn=<AddBackward0>) Loss: 0.05461053662002087 
Avg Training loss: tensor(1.3155, grad_fn=<AddBackward0>)
Avg Training loss: tensor(1.1955, grad_fn=<AddBackward0>)
Avg Training loss: tensor(1.1444, grad_fn=<AddBackward0>)
Avg Training loss: tensor(1.1159, grad_fn=<AddBackward0>)
Avg Training loss: tensor(1.0993, grad_fn=<AddBackward0>)
Avg Training loss: tensor(3.0410, grad_fn=<AddBackward0>) Loss: 0.07900961712002755 
Avg Training loss: tensor(1.6778, grad_fn=<AddBackward0>)
Avg Training loss: tensor(1.3988, grad_fn=<AddBackward0>)
Avg Training loss: tensor(1.2987, grad_fn=<AddBackward0>)
Avg Training loss:

KeyboardInterrupt: 

In [ ]:
print("Training complete")

In [ ]:
def meanScore(dataArr):
    meanModelData = []
    for i in range (len(dataArr)):
        meanScore = np.mean(dataArr[i])
        meanModelData.append(meanScore)
    return meanModelData

def minScore(dataArr):
    minModelScore = []
    for i in range (len(dataArr)):
        minScore = np.mean(dataArr[i])
        minModelScore.append(minScore)
    return minModelScore

def maxScore(dataArr):
    maxModelScore = []
    for i in range (len(dataArr)):
        maxScore = np.max(dataArr[i])
        maxModelScore.append(maxScore)
    return maxModelScore

In [ ]:
fig,ax=plt.subplots()
ax.plot(batc,minScore(modelsTrainLoss1),color="Blue", marker="o",linestyle='dashed')
ax.plot(batchArr,modelsTestLoss1,color="Blue", marker="v")
ax.legend(['Train Loss','Test Loss'],loc="upper right")
ax.set_xlabel("Batch",color="Green")
ax.set_ylabel("CrossEntropy Loss",color = "blue")
ax.set_title("Loss,Sensitivity Vs Batch size",color="red")

ax2=ax.twinx()
ax2.plot(batchArr,minScore(modelsSensitivityArr),color="hotpink")
ax2.set_xlabel("Batch",color="Green")
ax2.set_ylabel("Sensitivity",color = "hotpink")
ax2.legend(['Sensitivity'],loc="center left")
